# SeFa Implementation

## Fetch Codebase and Models

In [30]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), 'genforce'))
if os.path.exists('/content'):
    os.chdir('/content')
    CODE_DIR = 'SeFa'
    if not os.path.exists(CODE_DIR):
        !git clone https://github.com/MrLishu/GANInvert.git $CODE_DIR
    os.chdir(f'./{CODE_DIR}')

In [ ]:
MODEL_DIR = os.path.join('models', 'pretrain')
os.makedirs(MODEL_DIR, exist_ok=True)
!wget https://mycuhk-my.sharepoint.com/:u:/g/personal/1155082926_link_cuhk_edu_hk/EXqix_JIEgtLl1FXI4uCkr8B5GPaiJyiLXL6cFbdcIKqEA?e=WYesel\&download\=1 -O $MODEL_DIR/styleganinv_ffhq256_encoder.pth  --quiet
!wget https://mycuhk-my.sharepoint.com/:u:/g/personal/1155082926_link_cuhk_edu_hk/EbuzMQ3ZLl1AqvKJzeeBq7IBoQD-C1LfMIC8USlmOMPt3Q?e=CMXn8W\&download\=1 -O $MODEL_DIR/styleganinv_ffhq256_generator.pth  --quiet
!wget https://mycuhk-my.sharepoint.com/:u:/g/personal/1155082926_link_cuhk_edu_hk/EQJUz9DInbxEnp0aomkGGzAB5b3ZZbtsOA-TXct9E4ONqA?e=smtO0T\&download\=1 -O $MODEL_DIR/vgg16.pth  --quiet

## Import StyleGAN and GAN Inverter

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from genforce.utils.inverter import StyleGANInverter
from genforce.models.face_landmark_detector import FaceLandmarkDetector

inverter = StyleGANInverter('styleganinv_ffhq256', learning_rate=0.01, iteration=100,
                            reconstruction_loss_weight=1.0, perceptual_loss_weight=5e-5, regularization_loss_weight=0)
generator = inverter.G
resolution = inverter.G.resolution

def align(image_name):
    face_landmark_detector = FaceLandmarkDetector(resolution)
    face_infos = face_landmark_detector.detect(os.path.join('images', image_name))[0]
    image = face_landmark_detector.align(face_infos)
    return image

ModuleNotFoundError: No module named 'dlib'